In [ ]:
from pyspark import *;
from pyspark.sql import *;
from pyspark.sql.functions import *;
import pandas as pd;

In [ ]:
spark = SparkSession.builder.appName("Q8").getOrCreate()

df = spark.read.csv("HR_Analytics.csv",header=True,inferSchema=True)
df.printSchema()
df.show()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/HR_Analytics.csv.

In [ ]:
df.select([count(when(isnull(c)|isnan(c),c)).alias(c) for c in df.columns]).show()

In [ ]:
mean_distance = df.select(mean("DistanceFromHome")).collect()[0][0]
df = df.fillna({"BusinessTravel":"Unknown"})
df = df.fillna({"DistanceFromHome":mean_distance})
df = df.fillna({"EducationField":"Other"})

In [ ]:
df.select([count(when(isnull(c)|isnan(c),c)).alias(c) for c in df.columns]).show()

In [ ]:
df = df.dropDuplicates()

In [ ]:
df.select("DistanceFromHome").distinct().show()
df = df.filter(col("DistanceFromHome")<=50)
df.select("DistanceFromHome").distinct().show()

In [ ]:
df = df.withColumn("PercentSalaryHike_Category",
                   when(col("PercentSalaryHike")<=10, "Low").
                   when(col("PercentSalaryHike")<=20, "Moderate").
                   otherwise("High"))
df.select("PercentSalaryHike_Category").distinct().show()
df.select("PercentSalaryHike").distinct().show()

In [ ]:
df = df.withColumn("JobSatisfaction_Category",
                   when(col("JobSatisfaction")==1, "Poor").
                   when(col("JobSatisfaction")==2, "Below Average").
                   when(col("JobSatisfaction")==3, "Average").
                   when(col("JobSatisfaction")==4, "Excellent").
                   otherwise(None))
df.show()

In [ ]:
df = df.withColumn("SalarySlab", regexp_replace("SalarySlab","k","000"))
df = df.withColumn("SalarySlab", col("SalarySlab").cast("int"))
df.select("SalarySlab").distinct().show()

In [ ]:
df = df.toPandas()
df.to_csv("cleaned_8.csv")

In [ ]:
spark.stop()